<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLNN_1_1-160122.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Single Neuron

In [ ]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          bias:     The bias term. By defaul it's 1.0.
          weights:  List of weights incl. bias
          activ:    The activation function: linear (default), relu, mrelu (modified relu), sigmoid.
          param:    parameter of mrelu activ function -- slope of negatives. 0.01 by default."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.
        inputs:   The number of inputs in the perceptron, not counting the bias.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron according the activ function. x is a list with a single row of the input data."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight.
           w_init is a list of numbers. Turns to a numpy array of doubles."""
        self.weights=np.array(w_init, dtype='double')

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        """Returns the sigmoid of the input"""
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        """Returns the RELU of the input"""
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        """Returns the modified RELU of the input. The parameter is a slope for negatives"""
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

## AND gate

In [ ]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [ ]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [ ]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [ ]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: self.network[layer][neuron].set_activ('linear'). Layer 0 is an input.
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (0) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          network: self.network[layer][neuron] -- list of lists of Neurons. Layer 0 is an inputs.
          param:   Parameter of mrelu activ function -- slope of negatives. 0.01 by default.
          eta:     Learning rate."""

    def __init__(self, layers, bias = 1.0, activ='linear'):
        """Return a new MLP object with the specified parameters.
           layers -- a list of layers. [0] is a number of the model's features.
           Activation function is linear by default.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           Overrides the np.random.rand() weights and the bias weight.
           w_init -- a list of lists with the weights for all but the input layer. Incl. the bias. """
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons.
           activ -- a string of 'linear' (default), 'relu', 'mrelu' (modified relu), 'sigmoid'.
           param -- parameter for mrelu."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons.
           activ -- a string of 'linear' (default), 'relu', 'mrelu' (modified relu), 'sigmoid'.
           param -- parameter for mrelu."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a single row of x into the MultiLayer Neuron.
           Returns the output of the last neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y, eta=0.2):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
           Uses the derivative of the SIGMOID function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        self.eta=eta
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        """Return the output of the sigmoid function applied to x"""
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        '''Calculates the derivative of the activ function for the back propagation'''
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y, eta=0.01):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
           Uses the derivative according each neuron's activation function.
           Modifies the weights of the neurons, calculates and returns updated MSE.
           eta -- learning rate."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        self.eta=eta
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [ ]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10. -10.  15.] sigmoid
Layer 1 Neuron 1 : [ 15.  15. -10.] sigmoid
Layer 2 Neuron 0 : [ 10.  10. -15.] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [ ]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.0928440373229726
1.046078090551561
1.0428302938670928
1.0288932470886074
0.9234218877456317
0.7246144481545738
0.6303788449779767
0.594483676067328
0.5777366440257483
0.5684264630190407

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-3.4125294   6.00114049  2.69427592] sigmoid
Layer 1 Neuron 1 : [ 2.89923737  5.64759797 -0.55524576] sigmoid
Layer 2 Neuron 0 : [-4.53272966  4.3166207   0.13266529] sigmoid

XOR Gate:
0 0 = 0.0731662255
0 1 = 0.4727663057
1 0 = 0.9299330740
1 1 = 0.4845040435


# Regres

In [ ]:
class Regres:
    """Creates a multilayer neuron network.
       Used for regression. Fits the model by running the MultiLayer Neuron Network in a 
       loop for each row of X and calculating the error. Each running modifies the weights of the objects. 
       Attributes:
          layers:               A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          regres_network:       MultiLayerNeuron Class. self.regres_network.network[layer][neuron] -- Neuron Class (layer 0 is an input).
          epochs:               Number of iterations
          eta:                  Learning rate
          weight_history:       List of lists of weights propagation
          weight_history_table: Pandas table of weights propagation
          MSE_history:          List of MSEs propagation"""
      
    def __init__(self, layers, bias=1.0):
        """Return a new MLP object with the specified parameters.
           layers -- a list of layers. [0] is a number of the model's features.
           Activation function is linear by default.""" 
        self.layers = layers
        self.bias = bias
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias)

    def set_weights(self, w_init):
        """Set the weights. 
           Overrides the np.random.rand() weights and the bias weight.
           w_init -- a list of lists with the weights for all but the input layer. Incl. the bias. """
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        """Sets the activ function of the hidden layers.
           activ -- a string of 'linear' (default), 'relu', 'mrelu' (modified relu), 'sigmoid'.
           param -- parameter for mrelu."""
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs, eta=0.01):
        """Runs the MLNs epochs times. Each time the weights are being modified and the error is being calculated.
           MSEs and weights are stored.
           X,y -- an array and a list of data."""
        self.epochs=epochs
        self.eta=eta
        self.weight_history=[]
        self.weight_history_table=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            weight_epoch_table=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]], self.eta)
            MSE = MSE / len(X)
            self.MSE_history.append(MSE)
            for m in range(1,len(self.layers)):
                weight_layer=[]
                for n in range(self.layers[m]):
                    neuron_w=self.regres_network.network[m][n].weights
                    neuron_w_list=[x for x in neuron_w]
                    weight_layer.append(neuron_w_list)
                    weight_epoch_table+=neuron_w_list
                weight_epoch.append(weight_layer)
            self.weight_history.append(weight_epoch)
            self.weight_history_table.append(weight_epoch_table)
        self.weight_history_table=pd.DataFrame(data=self.weight_history_table, columns=self.get_cols())
        print ("""Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation""")

    def get_cols(self): 
        """Gets a list of names for weights. Used for pandas table of weights propagation as column names."""
        cols=[]
        for i in range(1,len(self.layers)):
            for h in range(self.layers[i]):
                for k in range(self.layers[i-1]+1): 
                    col="{}_{}_{}".format(i,h,k) 
                    cols.append(col)
        return cols 
    
    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        self.regres_network.printWeights()

    def run(self, x):
        """Calculates the output of a single row of X with the weights set"""
        return self.regres_network.run(x)

    def predict(self, x):
        """Returns the list of output. Runs every row of the data."""
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

In [ ]:
weight_history=[]
a=[[1,2],[1,3],[4,1]]
b=[]
for m in range(3):
  for n in range(2):
    b.append(a[m][n])
print ("b",b)
weight_history.append(b)
print (weight_history)

b [1, 2, 1, 3, 4, 1]
[[1, 2, 1, 3, 4, 1]]


# Example 1 -- weights

In [ ]:
print ("1 1 =",mln1.run([1,1]))
# print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"
w1=mln1.network[1][0].weights
w2=[x for x in w1]
w2

1 1 = [0.00715281]


[-10.0, -10.0, 15.0]

In [ ]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.16679103 -0.95485914 -0.39245372] relu
Layer 1 Neuron 1 : [-0.36852056  0.55979752  0.00982471] relu
Layer 2 Neuron 0 : [-0.21318853  0.38488427  0.55835809] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.16679103 -0.95485914 -0.39245372] relu
Layer 1 Neuron 1 : [-0.36852056  0.55979752  0.00982471] relu
Layer 2 Neuron 0 : [-0.21318853  0.38488427  0.55835809] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.16679103 -0.95485914 -0.39245372] linear
Layer 1 Neuron 1 : [-0.36852056  0.55979752  0.00982471] relu
Layer 2 Neuron 0 : [-0.21318853  0.38488427  0.55835809] sigmoid



# Example 3 -- linear activs, bp_regres. מתכנס למשקלים טוב מאוד 

In [ ]:
mln3 = MultiLayerNeuron(layers=[3,1])
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0,0,0,0]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

41.81534530017634
1.033154313956396e-16

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253295e-05] linear



# Example 4 -- sigmoid, bp_regres. מקרב רגרסיה עם סיגמויד. עובד טוב

In [ ]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] sigmoid
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] sigmoid
Layer 2 Neuron 0 : [-1.  2.  3.] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 14.560915350873733
MSE= 0.016587877280264497
MSE= 0.013933291819544657
MSE= 0.012140276381428039
MSE= 0.010836291305765211

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.47937599 -1.33871116  0.39961694  0.7452429 ] sigmoid
Layer 1 Neuron 1 : [ 0.38613108 -0.8072425   0.38862845 -0.11099338] sigmoid
Layer 2 Neuron 0 : [2.0141246  1.44754529 1.65136514] linear



'predicted values:'

[3.5113812595967566,
 3.6662245755626417,
 3.70087300117188,
 3.13667022572865,
 3.8964204127492303,
 2.2520609255857815]

# Example 5 -- relu-sigmoid, bp_regres. מתכנס למשקלים לא משהו

In [ ]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%20 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] relu
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] relu
Layer 2 Neuron 0 : [1. 0. 1.] sigmoid
Layer 3 Neuron 0 : [-1.  2.] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 4.557508180003647
MSE 20 = 0.03306422578001673
MSE 40 = 0.020833835690515373
MSE 60 = 0.007637349245875012
MSE 80 = 0.003019154384476364
MSE 100 = 0.0027154752605487373
MSE 120 = 0.0027614739025188825
MSE 140 = 0.002772116959020973
MSE 160 = 0.00276938831528622
MSE 180 = 0.0027649370609093217
MSE 200 = 0.0027612295422073906
MSE 220 = 0.002758488008391192
MSE 240 = 0.0027565271679670226
MSE 260 = 0.0027551422488969403
MSE 280 = 0.002754172799930343
MSE 300 = 0.0027535017890262955
MSE 320 = 0.002753045282328946
MSE 340 = 0.0027527432353624155
MSE 360 = 0.002752552650955737
MSE 380 = 0.0027524426948155743

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.8881536   0.32307107  0.87411481 -0.55865158] relu
Layer 1 Neuron 1 : [-0.05101864  1.03753757 -0.32346028 -0.47420628] relu
Layer 2 Neuron 0 : [ 0.8065893  -0.32603334 -0.02623846] sigmoid
Layer 3 Neuron 0 : [0.85532965 0.64780478] linear



'predicted values:'

[1.0377970742588682,
 1.263663582367751,
 1.0698592999194279,
 1.0342704984616755,
 1.0698592999194279,
 1.0051230931218662]

# Example 7 -- Regres class. עובד באופן עקרוני
# relu בעייתי

In [ ]:
reg1=Regres(layers=[3,2,1,1])
reg1.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=5, eta=0.1)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [9.99993917e-01 9.99950434e-01 1.19553645e-05 2.60048657e-06] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [0.00850294 0.         0.03500299] sigmoid
Layer 3 Neuron 0 : [0.42542099 0.84771537] linear

[1.0668595952299427, 1.0641482356911771, 1.06414823804216, 1.0659559852988165, 1.0641482356911771, 1.0641482356911771]



'MSE'

[1.8232957676324437,
 0.10904793902185193,
 0.04350360405968031,
 0.03926659710245326,
 0.03860509075063843]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [ ]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1])
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 0.37156705313996946
MSE 10 = 0.3477538356465295
MSE 20 = 0.2952678357523502
MSE 30 = 0.1929773244127845
MSE 40 = 0.08122195719845282
MSE 50 = 0.026313708157257025
MSE 60 = 0.009140606396217596
MSE 70 = 0.0030643254528590997
MSE 80 = 0.0009964697271421519
MSE 90 = 0.0004154365309422917

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.02456057  0.33840543  0.0334604   0.7232171   0.54268441 -0.05063808] sigmoid
Layer 1 Neuron 1 : [ 0.23164539  0.23135724 -0.27344665  1.21173095  0.55817302 -0.09354269] sigmoid
Layer 1 Neuron 2 : [ 0.89420685  0.80339977  1.20014113  0.31647156  0.44787927 -0.05550803] sigmoid
Layer 2 Neuron 0 : [ 0.492259    0.86010436 -0.75006689  0.11361553] linear



# Example 8 -- Regres_linear. עובד טוב

In [ ]:
reg2=Regres(layers=[1,2,1])
X=[[0],[1],[2],[3],[4],[5]]
y=[0,1,2,3,4,5]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5, eta=0.001)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
display ('weight_history_table',reg2.weight_history_table)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.11058018  0.14537331] linear
Layer 1 Neuron 1 : [-0.28297733  0.68056045] linear
Layer 2 Neuron 0 : [-0.1473272  -0.27336655  0.6430521 ] linear

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.17821406  0.12840375] linear
Layer 1 Neuron 1 : [-0.40612701  0.64964264] linear
Layer 2 Neuron 0 : [-0.19080044 -0.34323212  0.74622972] linear

predictions [0.4987520129516496, 0.6721511662275186, 0.8455503195033877, 1.0189494727792567, 1.1923486260551257, 1.3657477793309947]


'weight_history_table'

,1_0_0,1_0_1,1_1_0,1_1_1,2_0_0,2_0_1,2_0_2
0,-0.123420,0.142121,-0.306671,0.674555,-0.154056,-0.283572,0.664892
1,-0.136577,0.138804,-0.330769,0.668476,-0.161810,-0.295752,0.686161
2,-0.150085,0.135414,-0.355351,0.662303,-0.170552,-0.309822,0.706831
3,-0.163964,0.131947,-0.380465,0.656026,-0.180234,-0.325686,0.726868
4,-0.178214,0.128404,-0.406127,0.649643,-0.190800,-0.343232,0.746230


'MSE'

[22.673488232081507,
 21.724139386526602,
 20.72909733977657,
 19.681375852647687,
 18.57666311256642]

# Example 9 -- mrelu check. working

MultiLayerNeuron

In [ ]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

eta=0.01
for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0], eta)
    MSE += mln4.bp_regres([1],y4[1], eta)
    MSE += mln4.bp_regres([2],y4[2], eta)
    MSE += mln4.bp_regres([3],y4[3], eta)
    MSE += mln4.bp_regres([4],y4[4], eta)
    MSE += mln4.bp_regres([5],y4[5], eta)
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-0.38541999999999943,
 1.2342550000000005,
 2.7394859999999994,
 9.622496,
 16.505506,
 23.388515999999996]


MSE 0 = 632.3026639218954
MSE 200 = 35.7718087852277
MSE 400 = 38.39243043062413
MSE 600 = 34.204464583182364
MSE 800 = 27.722952207363445
MSE 1000 = 20.871627888396215
MSE 1200 = 15.61765685223395
MSE 1400 = 9.228759995625309
MSE 1600 = 4.235568801237803
MSE 1800 = 1.230352004985453
MSE 2000 = 0.12758309216043026
MSE 2200 = 0.11808008648499092
MSE 2400 = 0.10929856287368135
MSE 2600 = 0.10099282622731424
MSE 2800 = 0.09315818419030947
MSE 3000 = 0.08578679122004251
MSE 3200 = 0.07886860839146516
MSE 3400 = 0.07239165974398819
MSE 3600 = 0.06634229349227465
MSE 3800 = 0.06070544452675303

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10.77203854  -0.17985249] mrelu
Layer 1 Neuron 1 : [ 2.49938916 -4.78563572] mrelu
Layer 2 Neuron 0 : [-10.75684764   2.291674    -0.09006781] linear



'predicted values:'

[-0.18039252475862172,
 1.035617100058026,
 2.7351955598367756,
 9.62171248698544,
 16.508229414134107,
 23.394746341282772]

# Example 10 - Regres_mrelu. working

In [ ]:
reg2=Regres(layers=[1,2,1])
X=[[-3],[-2],[-1],[0],[1],[2],[3],[4],[5]]
y=[9,4,1,0,1,4,9,16,25]
# y=[0,1,2,3,4,5]
reg2.set_hidden_activ('mrelu', param=0.01)
# reg2.set_weights([[[0,0],[0,0]],[[0,0,0]]])
reg2.fit(X,y,epochs=5000, eta=0.0001)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
# display ('weight_history_table',reg2.weight_history_table.iloc[-1])
# display ('MSE',reg2.MSE_history)

fig=px.line(y=reg2.MSE_history[500:])
fig.show()

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.80316886 -3.58343932] mrelu
Layer 1 Neuron 1 : [ 0.11565218 -0.58600726] mrelu
Layer 2 Neuron 0 : [ 3.88770253 -0.82804783  2.92365254] linear

predictions [2.923652542163676, 2.923652542163676, 2.923652542163676, 2.923652542163676, 2.923652542163676, 3.0126747202921034, 10.02285885394936, 17.033042987606613, 24.04322712126387]


In [ ]:
history=reg2.weight_history_table.T
# history
fig=px.parallel_coordinates(history)
fig.show()